# UGR'16 Dataset Exploratory Data Analysis

This notebook performs exploratory data analysis on the UGR'16 network traffic dataset for anomaly detection research.

## Import Libraries

In [ ]:
# Install project in editable mode with all dependencies
%pip install -e ..

In [ ]:
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

In [ ]:
SAMPLE_PATH = os.getenv("SAMPLE_PATH")

In [ ]:
# Load the parquet sample directly into pandas
ugr_df_raw = pd.read_parquet(SAMPLE_PATH, engine='pyarrow')
print(f"Loaded {len(ugr_df_raw):,} rows from parquet sample.")

## Working with Pre-sampled Data

- This notebook uses a pre-sampled parquet file containing approximately 2.8 million flows, matching the CTU-13 working set size.
- The sample was created using DuckDB reservoir sampling with a fixed seed for reproducibility.
- All downstream EDA cells operate on the sampled DataFrame `ugr_df`.

In [ ]:
column_aliases = {
    "column0": "timestamp",
    "column00": "timestamp",
    "column1": "duration",
    "column01": "duration",
    "column2": "src_ip",
    "column02": "src_ip",
    "column3": "dst_ip",
    "column03": "dst_ip",
    "column4": "src_port",
    "column04": "src_port",
    "column5": "dst_port",
    "column05": "dst_port",
    "column6": "protocol",
    "column06": "protocol",
    "column7": "flags",
    "column07": "flags",
    "column8": "tos",
    "column08": "tos",
    "column9": "packets_fwd",
    "column09": "packets_fwd",
    "column10": "packets_bwd",
    "column11": "bytes_total",
    "column12": "label"
}
rename_map = {}
for original in ugr_df_raw.columns:
    if original in column_aliases:
        rename_map[original] = column_aliases[original]
ugr_df = ugr_df_raw.rename(columns=rename_map)
# Preserve any columns without aliases under their original names
print("Renamed columns:", rename_map)
print("Final columns:", ugr_df.columns.tolist())

In [ ]:
import numpy as np
import pandas as pd
import os

# Clear any conflicting matplotlib backend from environment
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']

import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend for notebooks
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_style("whitegrid")
plt.rcParams.update({"figure.figsize": (12, 6), "font.size": 10})

ugr_df['timestamp'] = pd.to_datetime(ugr_df.get('timestamp'))

numeric_casts = {
    'duration': float,
    'src_port': 'Int64',
    'dst_port': 'Int64',
    'packets_fwd': 'Int64',
    'packets_bwd': 'Int64',
    'bytes_total': float,
}
for col, dtype in numeric_casts.items():
    if col in ugr_df.columns:
        ugr_df[col] = pd.to_numeric(ugr_df[col], errors='coerce').astype(dtype)

categorical_columns = [c for c in ['protocol', 'flags', 'label'] if c in ugr_df.columns]
for col in categorical_columns:
    ugr_df[col] = ugr_df[col].astype('category')

malicious_keywords = ['botnet', 'attack', 'anomaly', 'malicious', 'ddos', 'worm', 'spam', 'blacklist']
if 'label' in ugr_df.columns:
    pattern = '|'.join(malicious_keywords)
    ugr_df['is_malicious'] = ugr_df['label'].astype(str).str.contains(pattern, case=False, na=False)
else:
    ugr_df['is_malicious'] = False

In [ ]:
summary_stats = {
    'rows': len(ugr_df),
    'time_range': (ugr_df['timestamp'].min(), ugr_df['timestamp'].max()) if 'timestamp' in ugr_df.columns else (None, None),
    'unique_src_ips': ugr_df['src_ip'].nunique() if 'src_ip' in ugr_df.columns else None,
    'unique_dst_ips': ugr_df['dst_ip'].nunique() if 'dst_ip' in ugr_df.columns else None,
    'unique_protocols': ugr_df['protocol'].nunique() if 'protocol' in ugr_df.columns else None,
    'unique_labels': ugr_df['label'].nunique() if 'label' in ugr_df.columns else None,
}
for key, value in summary_stats.items():
    print(f"{key}: {value}")

## Label Distribution and Class Imbalance

In [ ]:
label_counts = ugr_df['label'].value_counts(dropna=False).sort_values(ascending=False)
top_n = 10
top_labels = label_counts.head(top_n)
others = label_counts.iloc[top_n:].sum()
plot_counts = top_labels.copy()
if others > 0:
    plot_counts = pd.concat([plot_counts, pd.Series({'Others': others})])
label_percent = (plot_counts / len(ugr_df) * 100)

fig, axes = plt.subplots(1, 2, figsize=(20, 9))

# Enhanced color palette with better contrast
palette = []
for idx in plot_counts.index.astype(str):
    if 'bot' in idx.lower() or 'attack' in idx.lower():
        palette.append('#e63946')  # Red for malicious
    elif 'blacklist' in idx.lower() or 'anomaly' in idx.lower():
        palette.append('#f77f00')  # Orange for suspicious
    else:
        palette.append('#06a77d')  # Green for background

# Left panel: Horizontal bar chart with better formatting
bars = axes[0].barh(range(len(plot_counts)), plot_counts.values, color=palette, edgecolor='black', linewidth=0.8)
axes[0].set_yticks(range(len(plot_counts)))
axes[0].set_yticklabels(plot_counts.index.astype(str), fontsize=11, fontweight='bold')
axes[0].set_xlabel('Flow Count', fontsize=12, fontweight='bold')
axes[0].set_title(f'Top {top_n} Labels + Others', fontsize=14, fontweight='bold', pad=15)
axes[0].grid(axis='x', alpha=0.3, linestyle='--')
axes[0].invert_yaxis()

# Add value labels with better positioning
for i, (bar, count, pct) in enumerate(zip(bars, plot_counts.values, label_percent.values)):
    axes[0].text(count + max(plot_counts.values)*0.01, i, 
                 f'{count:,.0f} ({pct:.1f}%)', 
                 va='center', fontsize=10, fontweight='bold')

# Right panel: Improved pie chart with better labels
def make_autopct(values):
    def my_autopct(pct):
        if pct > 1.0:  # Only show percentages > 1%
            return f'{pct:.1f}%'
        return ''
    return my_autopct

wedges, texts, autotexts = axes[1].pie(
    plot_counts.values, 
    labels=None,
    autopct=make_autopct(plot_counts.values),
    startangle=90, 
    colors=palette,
    wedgeprops={'edgecolor': 'black', 'linewidth': 1.2}
)

# Style the percentage text
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontsize(11)
    autotext.set_fontweight('bold')

axes[1].set_title('Distribution by Percentage', fontsize=14, fontweight='bold', pad=15)

# Create a better legend
legend_labels = [f'{label}: {count:,.0f} ({pct:.1f}%)' 
                 for label, count, pct in zip(plot_counts.index.astype(str), 
                                               plot_counts.values, 
                                               label_percent.values)]
axes[1].legend(wedges, legend_labels, 
               title='Flow Labels', 
               title_fontsize=11,
               fontsize=9,
               loc='center left', 
               bbox_to_anchor=(1.05, 0.5),
               frameon=True,
               edgecolor='black')

fig.suptitle('UGR\'16 Label Distribution (Sampled: 2.8M Flows)', 
             fontsize=16, fontweight='bold', y=0.98)
fig.tight_layout()

figures_dir = Path('../docs/figures')
figures_dir.mkdir(parents=True, exist_ok=True)
fig.savefig(figures_dir / 'ugr16_label_distribution.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.close(fig)

tables_dir = Path('../docs/tables')
tables_dir.mkdir(parents=True, exist_ok=True)

label_summary = (
    ugr_df.groupby('label').agg(
        Flows=('label', 'size'),
        Flow_Percentage=('label', lambda x: len(x) / len(ugr_df) * 100),
        Avg_Duration_s=('duration', 'mean'),
        Avg_Total_Packets=('packets_fwd', 'mean'),
        Avg_Total_Bytes=('bytes_total', 'mean')
)
    .sort_values('Flows', ascending=False)
    .reset_index()
    .rename(columns={'label': 'Label'})
)
label_summary.head(20).to_latex(
    tables_dir / 'ugr16_label_summary_top20.tex',
    index=False,
    longtable=True,
    float_format='{:,.2f}'.format,
    caption='Top 20 Labels by Flow Count (UGR\'16 Sample)',
    label='tab:ugr16_label_summary'
)

## Protocol and Port Characteristics

In [ ]:
protocol_counts = ugr_df['protocol'].value_counts().head(10) if 'protocol' in ugr_df.columns else pd.Series(dtype=int)
top_ports = ugr_df['dst_port'].value_counts().head(20) if 'dst_port' in ugr_df.columns else pd.Series(dtype=int)

fig, axes = plt.subplots(1, 2, figsize=(16, 7))
protocol_counts.plot(kind='bar', ax=axes[0], color='#339af0')
axes[0].set_title('Top Protocols')
axes[0].set_ylabel('Flows')
axes[0].set_xlabel('Protocol')
axes[0].grid(axis='y', alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

axes[1].barh(top_ports.index.astype(str), top_ports.values, color='#845ef7')
axes[1].invert_yaxis()
axes[1].set_title('Top Destination Ports')
axes[1].set_xlabel('Flows')
axes[1].grid(axis='x', alpha=0.3)

for y, value in enumerate(top_ports.values):
    axes[1].text(value, y, f'{value:,.0f}', va='center')

fig.tight_layout()
fig.savefig(figures_dir / 'ugr16_protocol_port_profiles.png', dpi=300, bbox_inches='tight')
plt.close(fig)

protocol_table = (
    ugr_df.groupby('protocol').size().sort_values(ascending=False).reset_index(name='Flows')
    if 'protocol' in ugr_df.columns
    else pd.DataFrame(columns=['protocol', 'Flows'])
)
if 'protocol' in ugr_df.columns:
    protocol_table.to_latex(
        tables_dir / 'ugr16_protocol_summary.tex',
        index=False,
        caption='Protocol Frequency Summary (UGR\'16 Sample)',
        label='tab:ugr16_protocol_summary'
)
if 'dst_port' in ugr_df.columns:
    top_ports.reset_index().rename(columns={'index': 'Destination Port', 'dst_port': 'Flows'}).to_latex(
        tables_dir / 'ugr16_top_ports.tex',
        index=False,
        caption='Top 20 Destination Ports (UGR\'16 Sample)',
        label='tab:ugr16_top_ports'
)

## Attack vs Background Flow Characteristics

In [ ]:
if 'is_malicious' in ugr_df.columns and ugr_df['is_malicious'].any():
    comparison_features = [
        ('duration', 'Duration (s)'),
        ('packets_fwd', 'Forward Packets'),
        ('packets_bwd', 'Backward Packets'),
        ('bytes_total', 'Total Bytes')
]
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    for ax, (feature, label) in zip(axes, comparison_features):
        if feature not in ugr_df.columns:
            ax.set_visible(False)
            continue
        upper = ugr_df[feature].dropna().quantile(0.99)
        bot = ugr_df.loc[ugr_df['is_malicious'], feature].dropna().clip(upper=upper)
        back = ugr_df.loc[~ugr_df['is_malicious'], feature].dropna().clip(upper=upper)
        bins = np.linspace(0, upper, 60) if upper and np.isfinite(upper) and upper > 0 else 50
        ax.hist([bot, back], bins=bins, label=['Malicious', 'Benign'],
                color=['#ff8787', '#69db7c'], alpha=0.7, edgecolor='black')
        ax.set_title(label)
        ax.set_xlabel(label)
        ax.set_ylabel('Flows (log)')
        ax.set_yscale('log')
        ax.legend()
        ax.grid(alpha=0.3)

    fig.suptitle('UGR\'16 Malicious vs Benign Flow Characteristics (capped at 99th percentile)')
    fig.tight_layout()
    fig.savefig(figures_dir / 'ugr16_attack_vs_background.png', dpi=300, bbox_inches='tight')
    plt.close(fig)

    comparison_table = (
        ugr_df.groupby('is_malicious').agg(
            flows=('is_malicious', 'size'),
            avg_duration=('duration', 'mean'),
            avg_packets_fwd=('packets_fwd', 'mean'),
            avg_packets_bwd=('packets_bwd', 'mean'),
            avg_bytes=('bytes_total', 'mean')
)
        .rename(index={True: 'Malicious', False: 'Benign'})
)
    comparison_table.to_latex(
        tables_dir / 'ugr16_attack_vs_background.tex',
        float_format='{:,.2f}'.format,
        caption='Malicious vs Benign Flow Statistics (UGR\'16 Sample)',
        label='tab:ugr16_attack_background'
)
else:
    print('Warning: No malicious flows detected in the sampled subset using the current keyword list.')

## Temporal Activity Patterns

In [ ]:
if 'timestamp' in ugr_df.columns:
    hourly_counts = ugr_df.set_index('timestamp').resample('1H').size()
    fig, ax = plt.subplots(figsize=(16, 5))
    hourly_counts.plot(ax=ax, color='#1098ad')
    ax.set_title('Hourly Flow Volume (Sampled)')
    ax.set_xlabel('Hour')
    ax.set_ylabel('Flows per Hour')
    ax.grid(alpha=0.3)
    fig.tight_layout()
    fig.savefig(figures_dir / 'ugr16_hourly_activity.png', dpi=300, bbox_inches='tight')
    plt.close(fig)

    if 'is_malicious' in ugr_df.columns and ugr_df['is_malicious'].any():
        hourly_label = (
            ugr_df.set_index('timestamp')
            .groupby('is_malicious')
            .resample('1H')
            .size()
            .unstack(0)
            .fillna(0)
)
        fig, ax = plt.subplots(figsize=(16, 5))
        hourly_label.plot(ax=ax, color=['#69db7c', '#ff8787'])
        ax.set_title('Hourly Volume by Malicious Flag')
        ax.set_xlabel('Hour')
        ax.set_ylabel('Flows per Hour')
        ax.grid(alpha=0.3)
        fig.tight_layout()
        fig.savefig(figures_dir / 'ugr16_hourly_activity_by_label.png', dpi=300, bbox_inches='tight')
        plt.close(fig)

    temporal_stats = (
        hourly_counts.describe().to_frame(name='Flows per Hour')
)
    temporal_stats.to_latex(
        tables_dir / 'ugr16_temporal_stats.tex',
        float_format='{:,.2f}'.format,
        caption='Hourly Flow Volume Summary (UGR\'16 Sample)',
        label='tab:ugr16_temporal_stats'
)
else:
    print('Timestamp column not available; skipping temporal analysis.')

### Notes on UGR'16 Sample for C2 Beaconing Experiments
- UGR'16 mixes benign ISP traffic with diverse malicious events (blacklists, botnets, scans).
- The sampled subset mirrors the CTU workload in size, enabling fair comparisons for federated training.
- Labels often encode the detection source (e.g., blacklist, botnet, compromised server); adjust `malicious_keywords` to refine the positive class if needed.
- Heterogeneous protocols and destination ports highlight the need for robust feature engineering (temporal variance, burstiness, entropy).

In [ ]:
# Summary Statistics
print("=" * 80)
print("UGR'16 SAMPLE SUMMARY STATISTICS")
print("=" * 80)
print()
if 'timestamp' in ugr_df.columns:
    print(f"Time Range: {ugr_df['timestamp'].min()} to {ugr_df['timestamp'].max()}")
print(f"Total flows in sample: {len(ugr_df):,}")
if 'src_ip' in ugr_df.columns:
    print(f"Unique source IPs: {ugr_df['src_ip'].nunique():,}")
if 'dst_ip' in ugr_df.columns:
    print(f"Unique destination IPs: {ugr_df['dst_ip'].nunique():,}")
if 'protocol' in ugr_df.columns:
    print(f"Unique protocols: {ugr_df['protocol'].nunique()}")
if 'label' in ugr_df.columns:
    print(f"Unique labels: {ugr_df['label'].nunique()}")
print()
numeric_cols = [col for col in ['duration', 'packets_fwd', 'packets_bwd', 'bytes_total'] if col in ugr_df.columns]
if numeric_cols:
    display(ugr_df[numeric_cols].describe(percentiles=[0.5, 0.9, 0.99]).T.style.format('{:,.2f}').set_caption('Numeric Feature Summary'))

In [ ]:
numeric_cols = [col for col in ['duration', 'packets_fwd', 'packets_bwd', 'bytes_total'] if col in ugr_df.columns]
if numeric_cols:
    corr = ugr_df[numeric_cols].corr()
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', ax=ax, square=True, cbar_kws={'shrink': 0.7})
    ax.set_title('Correlation Matrix (UGR\'16 Sample)')
    fig.tight_layout()
    fig.savefig(figures_dir / 'ugr16_numeric_correlation.png', dpi=300, bbox_inches='tight')
    plt.close(fig)
else:
    print('Not enough numeric columns for correlation heatmap.')